# Sentiment Analysis

### Movie Review Dataset

This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset

In [15]:
import tensorflow as tf
import os
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from tensorflow import keras

In [2]:
VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
len(train_data[0])

218

### More preprocessing

We can see that the reviews are in different lengths, but we cannot pass different length data into our neural network. Therefore we must make each review the same length.
- If the review is greater than 250 words then trim off the extra words.
- If the review is less than 250 words add the necessary amount of 0's to make it equal to 250

In [9]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

### Creating the model

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32), #transforma palabras en vectores de numeros reales que capturan relacioones semanticas entre palabras
    tf.keras.layers.LSTM(32), #capa recurrente capaz de aprender dependencias a largo plazo de secuencias de texto
    tf.keras.layers.Dense(1, activation='sigmoid') #capa densa (totalmente conectada), la funcion de activacion sigmoide comprime la salida en el rango [0,1] lo cual es ideal para tareas de clasificacion binaria
    #el modelo luego puede decidir si una instancia pertenece a una clase especifica comparando esta probabilidad con un umbral, tipicamente 0.5
])

### Training

In [26]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
history = model.fit (train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 23s 35ms/step - acc: 0.9808 - loss: 0.0629 - val_acc: 0.8804 - val_loss: 0.3778
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - acc: 0.9829 - loss: 0.0542 - val_acc: 0.8800 - val_loss: 0.4008
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - acc: 0.9869 - loss: 0.0403 - val_acc: 0.8496 - val_loss: 0.5522
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - acc: 0.9874 - loss: 0.0418 - val_acc: 0.8816 - val_loss: 0.4550
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - acc: 0.9916 - loss: 0.0306 - val_acc: 0.8790 - val_loss: 0.4630
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - acc: 0.9928 - loss: 0.0220 - val_acc: 0.8738 - val_loss: 0.4838
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - acc: 0.9945 - loss: 0.0186 - val_acc: 0.8788 - val_loss: 0.5685
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - acc: 0.9942 - loss: 0.0195 - val_acc: 0.8704 - val_loss: 0.4830
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 

In [27]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - acc: 0.8092 - loss: 1.0500
[1.0269591808319092, 0.811519980430603]


We evaluated the model to see how well it performs. 

### Making predictions

We will use the network to make predictions on our own reviews. Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well , we load the encodings from the dataset and use them to encode our own data. 

ENCODING

In [28]:
word_index = imdb.get_word_index()

def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

text = 'that movie was just amazing, so amazing'
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

DECODING

In [29]:
# decode function 
#convierte la secuencia de enteros de vuelta a su forma textual

reverse_word_index = {value: key for (key, value) in word_index.items()} #invierte word_index
def decode_integers (integers): 
    PAD = 0 #relleno, el codigo omite 0 en la decodificacion
    text = ""
    for num in integers:
        if num != PAD: #busca la palabra correspondiente en "reverse_word_index" y la agrega en la cadena text
            text += reverse_word_index[num] + " "
    return text[:-1] #devuelve el texto codificado, quitando el ultimo espacio agregado durante la concatenacion

print (decode_integers(encoded))


that movie was just amazing so amazing


Now time to make a prediction

In [30]:
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250)) #crea un array de Numpy de ceros con forma (1,250). Aqui 1 representa el numero de muestras y 250 es la longitud maxima de la secuencia de texto
    pred[0] = encoded_text # Coloca la secuencia codificada en el primer (y único) ejemplo del array pred. Esto es necesario porque los modelos en Keras esperan que los datos de entrada tengan una forma (batch_size, sequence_length), incluso si el batch size es 1.
    result = model.predict(pred) #usa el modelo preentrenado para hacer la prediccion
    print(result[0]) #array de una sola probabilidad

positive_review = 'That movie was so awesome! I really loved it and would watch it again becuse it was amazingly great'
predict(positive_review)

negative_review = 'that movie sucked. I hated it and would not watch it again. Was one of the worst things I have ever watched '
predict(negative_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
[0.9505496]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[0.3101921]
